In [ ]:
import pandas as pd
import pyodbc
import warnings
from datetime import datetime, timedelta
import numpy as np
import warnings

warnings.filterwarnings('ignore')

# Connessione al database e estrazione dei dati
cnxn2 = pyodbc.connect(r'Driver=SQL Server;Server=dwhbisqltst;Database=SCM_INTEGRATION;Trusted_Connection=yes;')
Status = pd.read_sql("""
    SELECT sdhMachineId, data_value, CONVERT(datetime, [timestamp]) AS [from]
    FROM [SCM_INTEGRATION].[dbo].[INT_CR_Measure_hot_MachineStatus]
    WHERE 1=1 AND [timestamp_date] > DATEADD(DAY, -5, GETDATE())
    ORDER BY sdhMachineId, [timestamp] desc
""", cnxn2)

# Conversione della colonna 'from' in datetime
Status['from'] = pd.to_datetime(Status['from'].astype(str).str[:19], format='%Y-%m-%d %H:%M:%S')

# Ordinamento dei dati
Status.sort_values(by=['sdhMachineId', 'from'], ascending=[True, True], inplace=True)


print(Status.to_string(max_cols=Status.shape[1]))

#


In [ ]:

# Aggiunta delle righe di apertura e chiusura
max_from = Status['from'].max()
grouped_status = Status.groupby('sdhMachineId')

new_data = []

for name, group in grouped_status:
    min_from = group['from'].min()
    new_data.append({'sdhMachineId': name, 'data_value': 'UPDATE', 'from': min_from})
    new_data.extend(group.to_dict(orient='records'))
    new_data.append({'sdhMachineId': name, 'data_value': 'UPDATE', 'from': max_from})

Status_Machine = pd.DataFrame(new_data)

# Creazione della colonna 'to'
Status_Machine['to'] = Status_Machine.groupby('sdhMachineId')['from'].shift(-1)
Status_Machine['to'].fillna(Status_Machine['from'], inplace=True)

print(Status_Machine.to_string(max_cols=Status_Machine.shape[1]))


In [ ]:

import pandas as pd
from datetime import datetime, timedelta

# Funzione per dividere gli eventi giornalieri
def split_events_daily(start, end):
    events = []
    end_of_day = datetime(start.year, start.month, start.day, 23, 59, 59)
    while start < end:
        if start.date() == end.date():
            events.append((start, end))
            break
        else:
            events.append((start, end_of_day))
            start = end_of_day + timedelta(seconds=1)
            end_of_day = datetime(start.year, start.month, start.day, 23, 59, 59)
    return events

Status_Machine_split = []

for _, row in Status_Machine.iterrows():
    events = split_events_daily(row['from'], row['to'])
    for event in events:
        Status_Machine_split.append({'sdhMachineId': row['sdhMachineId'], 'data_value': row['data_value'], 'from': event[0], 'to': event[1]})

Status_Machine_split = pd.DataFrame(Status_Machine_split)

# Imposta le opzioni di visualizzazione per Pandas
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Stampa il DataFrame risultante
print(Status_Machine_split.to_string(index=False))
